# Ein anharmonischer Oszillator

## Symbolische und numerische Lösung eines physikalischen Systems

Wir wollen uns mit SymPy, NumPy und SciPy die Entwicklung eines Oszilatorsystems ansehen, welches analytisch nicht mehr komplett behandelt werden kann.

<img src="figs/Anharmonischer_Oszillator.png" style="margin:25px;" align="left" width=300 />

<br />

  Vektoren:
  
  - $\vec{e}_1=\begin{pmatrix} 1 \\ 0 \end{pmatrix}$, $\vec{e}_2=\begin{pmatrix} 0 \\ 1 \end{pmatrix}$
  - $\vec{r}_1 = P-P_1$, $\vec{r}_2 = P-P_2$
  - $\vec{e}_{r1} = \frac{\vec{r}_1}{|\vec{r}_1|}$, $\vec{e}_{r2} = \frac{\vec{r}_2}{|\vec{r}_2|}$

  Federauslenkung und Federkraft:
  
  - $\Delta_1 = |\vec{r}_1|-l_1$, $\Delta_2 = |\vec{r}_2|-l_2$
  - $\vec{F}_1 = -k_1\Delta_1\vec{e}_{r1}$, $\vec{F}_2 = -k_2\Delta_2\vec{e}_{r2}$
  - $\vec{F} = \vec{F}_1 + \vec{F}_2\qquad\text{ (Gesamtkraft)}$

  Bewegungsgleichungen:

  $\boxed{m\begin{pmatrix} \ddot{x_1} \\ \ddot{x_2} \end{pmatrix} = \begin{pmatrix} \vec{F}\cdot\vec{e}_1 \\ \vec{F}\cdot\vec{e}_2 \end{pmatrix}}$

In [ ]:
%matplotlib inline

# Symbolisches Aufstellen der Bewegungsgleichunbgen
import sympy as sp

# Numerische Lösung der Bewegungsgleichungen 
import numpy as np
import scipy.integrate as si
import matplotlib.pyplot as plt

# Animation des Oszillators
from matplotlib import animation

# Nötiges Modul, um die Federn in den Animationen 
# zu zeichnen (eigenes Modul)
import spring

## 1. Symbolische Herleitung der Bewegungsgleichungen mit SymPy

### 1.1. Definiere SymPy-Variable für das Federsystem

In [ ]:
# Größen für das Federsystem
m, l1, l2, k1, k2 = sp.symbols('m, l_1, l_2, k_1, k_2', real=True, positive=True)

t = sp.symbols('t', real=True)
x1, x2 = sp.symbols('x_1, x_2', cls=sp.Function, real=True)
x1 = x1(t)
x2 = x2(t)

### 1.2. Definiere die notwendigen Vektorgrößen

In [ ]:
## SOLUTION

# Definition der Vektoren
#
e1 = sp.Matrix([1, 0])
e2 = sp.Matrix([0, 1])

P = sp.Matrix([x1, x2])
P1 = sp.Matrix([-l1, 0])
P2 = sp.Matrix([0, -l2])

r1 = P - P1
r2 = P - P2

er1 = r1 / r1.norm()
er2 = r2 / r2.norm()

### 1.3. Definiere Federauslenkungen und Federkraft

In [ ]:
## SOLUTION

# Auslenkung des Federsystems und die Federkraft
Delta1 = r1.norm() - l1
Delta2 = r2.norm() - l2

F1 = -k1 * Delta1 * er1
F2 = -k2 * Delta2 * er2

F = F1 + F2
F

### 1.4. Stelle die Bewegungsgleichungen auf (symbolisch)

In [ ]:
## SOLUTION

# Die Bewegungsgleichungen

# symbolisch
x1_dd = F.dot(e1) / m
x2_dd = F.dot(e2) / m
sp.Eq(sp.diff(x1, t, 2), x1_dd)

## 2. Numerische Lösung der Bewegungsgleichungen

### 2.1. Umwandlung der symbolischen Bewegungsgleichungen in NumPy-Funktionen

In [ ]:
## SOLUTION

# Numerische Version der Bewegungsgleichungem
x1_dd_n = sp.lambdify([x1, x2, m, k1, k2, l1, l2], x1_dd)
x2_dd_n = sp.lambdify([x1, x2, m, k1, k2, l1, l2], x2_dd)

### 2.2. Implementation der Bewegungsgleichungen in `odeint`

Transformiere die Bewegungsgleichungen 2. Ordnung

$$
\ddot{x_1} = f_1(x_1, x_2, \dots)\qquad\text{und}\qquad\ddot{x_2} = f_2(x_1, x_2, \dots)
$$

in ein System aus gekoppelten Gleichungen erster Ordnung:

\begin{align*}
  \dot{x_1} & = u \\
  \dot{x_2} & = v \\
  \dot{u} & = f_1(x_1, x_2, \dots) \\
  \dot{v} & = f_2(x_1, x_2, \dots) \\
\end{align*}

Bringe das System mit $\vec{S}=(x_1, x_2, u, v)$ in Vektorform:

$$ \vec{S} = \begin{bmatrix} x_1(t) \\ x_2(t) \\ u(t) \\ v(t) \end{bmatrix} \hspace{5mm} \implies  \hspace{5mm} \frac{\rm{d}\vec{S}}{\rm{d}t} = \begin{bmatrix} \dot{x_1}(t) \\ \dot{x_2}(t) \\ \dot{u}(t) \\ \dot{v}(t) \end{bmatrix} = \begin{bmatrix} u(t) \\ v(t) \\ f_1(x_1, x_2, \dots) \\ f_2(x_1, x_2, \dots) \end{bmatrix}$$



Definiere eine Python-Funktion mit Argumenten (S, t, ...) und Rückgabewert ${\rm d}\vec{S}/{\rm d}t$:

In [ ]:
## SOLUTION

def dSdt(S, t, m, k1, k2, l1, l2):
    x1, x2, u, v = S
    
    dx1dt = u
    dx2dt = v
    dudt = x1_dd_n(x1, x2, m, k1, k2, l1, l2)
    dvdt = x2_dd_n(x1, x2, m, k1, k2, l1, l2)
    
    return [dx1dt, dx2dt, dudt, dvdt]

Definiere Zeiten, an denen das DGL-System gelöst werden soll, die Anfangsbedingung und nötige Konstanten. Zuletzt mit allem die DGL-Routine `odeint` aufrufen:

In [ ]:
## SOLUTION

# Zeit. Wir wollen eine Simulation von 100 s mit
# 10 Datenpunkten pro Sekunde. Das nutzen wir unten
# in der Animation
#

time = 100 
sampling = 10
t_n = np.linspace(0.0, time, time * sampling + 1)

# Konstanten
m_n, k1_n, k2_n, l1_n, l2_n = 1., 1., 1., 1., 1.

# Einige Anfangsbedingungen mit interessanten
# Bewegungsmustern:

#x1_0, x2_0, u0, v0 = 0.5, 0.0, 0.0, 0.0
x1_0, x2_0, u0, v0 = 0.7, 0.3, 0.0, 0.0
#x1_0, x2_0, u0, v0 = 0.7, 0.3, -0.5, 0.5
#x1_0, x2_0, u0, v0 = 0.0, -0.95, 0.0, 0.0
#x1_0, x2_0, u0, v0 = 0.0, 0.0, 0.5, 0.1
#x1_0, x2_0, u0, v0 = 0.0, 0.0, 2.0, 0.0

S0 = [x1_0, x2_0, u0, v0]

# und die Loesung des DGL-Systems
sol_dgl = si.odeint(dSdt, S0, t=t_n, args=(m_n, k1_n, k2_n, l1_n, l2_n))

## 3. Visualisiere Ergebnisse

### 3.1. Einfache Plots

In [ ]:
## SOLUTION

x1_n, x2_n = sol_dgl[:,0], sol_dgl[:,1]
plt.plot(x1_n, x2_n, label=f'S0={S0}')
plt.legend()

### 3.2. Animation der Pendelbewegungen

Die folgenden Animationen erzeugen *bewegte* gif-Dateien. Bitte konsultieren Sie das WWW, wenn Sie andere Formate (z.B. mp4) möchte, oder wenn der Code bei Ihnen nicht funktioniert.

In [ ]:
# Die Animate-Funktion ist für die einzelnen Bilder
# des Films zuständig:
def animate(i, ln1, ln2):
    ln1.set_data([x1_n[i]], [x2_n[i]])
    ln2.set_data(x1_n[0:i], x2_n[0:i])

    return ln1, ln2

fig, ax = plt.subplots(1, 1, figsize=(5, 5))

# Das erste Bild der Animation
ln1, = plt.plot([], [], lw=3, markersize=20, marker='o', 
                color='black', markerfacecolor='lightblue')
ln2, = plt.plot([], [], lw=2)

ax.set_ylim(-3.0, 2.0)
ax.set_xlim(-3.0, 2.0)
ax.grid()

# Die Anzahl der Frames entspricht der Anzahl der Zeitschritte der
# Simulation
ani = animation.FuncAnimation(fig, animate, fargs=(ln1, ln2), frames=time * sampling + 1)
plt.close()

# Wir haben insgesamt 1000 Zeitschritte bzw. Bilder. Mit fps=50
# (frames per second) erhalten wir zum Schluss eine Simulation von 20 Sekunden
ani.save(f'pendulum_{S0[0]}_{S0[1]}_{S0[2]}_{S0[3]}.gif', writer='pillow', fps=50)

!! SOLUTION

### 3.3. Animation der Pendelbewegungen (mit Federsystem)

In [ ]:
## SOLUTION

def animate(i, ln1, ln2, ln3, ln4, label, fast_forward):
    # zeichne Masse und Massenspur:
    ln1.set_data([x1_n[i]], [x2_n[i]])
    ln2.set_data(x1_n[0:i], x2_n[0:i])

    # zeichne die Federn:
    s1 = spring.spring([-l1_n, 0.], [x1_n[i], x2_n[i]], 10, 0.2)
    s2 = spring.spring([0., -l2_n], [x1_n[i], x2_n[i]], 10, 0.2)
    ln3.set_data(*s1)
    ln4.set_data(*s2)

    label.set_text(f"Zeit: {i / sampling / fast_forward:05.2f} / {time / fast_forward} s; S0={S0}")
    
    return ln1, ln2, ln3, ln4, label

fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.grid()

# ln1: Die Masse
ln1, = plt.plot([], [], lw=3, markersize=30, marker='o', 
                color='black', markerfacecolor='lightblue')
# ln2: Die Massenspur
ln2, = plt.plot([], [], lw=2)

# ln3, ln4: Federn:
ln3, = plt.plot([], [], lw=3, color='black', alpha=0.5)
ln4, = plt.plot([], [], lw=3, color='black', alpha=0.5)

label = ax.text(-2.9, 2.2, f"Zeit 0 / {time}", 
                ha='left', va='bottom', fontsize=15, color="black")

# zeichne Gleichgewichtspunkte:
ax.plot([-l1_n, 0], [0, -l2_n], color='red', lw=2)
ax.plot([0], [-l2_n], color='red', lw=2)

ax.set_ylim(-3.0, 2.0)
ax.set_xlim(-3.0, 2.0)

# Filmbeschleunigung
fast_forward = 5

# Erstelle den Film:
ani = animation.FuncAnimation(fig, animate, fargs=(ln1, ln2, ln3, ln4, label, fast_forward),
                              frames=time * sampling, interval=50, repeat=False)
plt.close()

ani.save(f'pendulum_{S0[0]}_{S0[1]}_{S0[2]}_{S0[3]}_feder.gif', writer='pillow', fps=sampling * fast_forward)